In [136]:
import numpy as np
from scipy import sparse
from scipy import linalg as la
import networkx as nx
import itertools
from matplotlib import pyplot as plt
import seaborn
%matplotlib inline

# Documentation Notes and Implementation Decisions

- User is not allowed to give an initial y0 value so we are guaranteed to satisfy the conditions of Theorem 2.1
- Initial x0 value is optional and defaults to all ones.
- Maximum iterations and tolerance values are keyword arguments.
- Use vector update version because scipy sparse matrices can do matrix-vector multiplication efficiently, but with the matrix updating you're multiplying a dense matrix with a sparse matrix a LOT.
- Testing revealed that the A.max(empty) and A.min(empty) method for extracting the source and terminus edge matrices from the incidence matrix are up to twice as fast as the "0.5(A +/- abs(A))" method for large sparse graphs, although they are similar for smaller graphs.
- TODO: Make a class that includes the n, m, At, As, and nice print functions.

In [127]:
class simscore(object):
    """
    Calculate similarity scores between nodes in G_A and G_B using the 
    node-edge scoring method of Zager and Verghese.
    
    G_A and G_B are NetworkX graphs.
    """
    
    def __init__(self, G_A, G_B, x0 = None, tol=1e-6, maxiters=10):
        
        self.G_A = G_A
        self.G_B = G_B
        self.x0 = x0
        self.y0 = None
        self.tol = tol
        self.maxiters = maxiters
        
        # Get the order and size of each graph
        self.n_A, self.m_A = G_A.number_of_nodes(), G_A.number_of_edges()
        self.n_B, self.m_B = G_B.number_of_nodes(), G_B.number_of_edges()
        
        
    def _initialize(self):
        """
        Get initial conditions and calculate the update matrix G if
        method == 'vector'. 
        
        If x0 is not specified (that is, no initial similarity information
        is known), both x0 and y0 are initialized as all ones.
        """
        
        # Get the source and terminus edge matrices for each graph
        A = nx.incidence_matrix(self.G_A, oriented=True)
        spA = sparse.csr_matrix(A.shape)
        self.At = A.maximum(spA)
        self.As = A.minimum(spA)
        
        B = nx.incidence_matrix(self.G_B, oriented=True)
        spB = sparse.csr_matrix(B.shape)
        self.Bt = B.maximum(spB)
        self.Bs = B.minimum(spB)
            
        self.G_T = sparse.kron(self.As, self.Bs) + sparse.kron(self.At, self.Bt)
        self.G = self.G_T.transpose()

        if self.x0 is None:
            self.x0 = np.ones(self.n_B*self.n_A)
            self.y0 = np.ones(self.m_B*self.m_A)

        else:
            self.y0 = self.G.dot(self.x0)
    
    def _update(self, x, y):
        
        x_new = self.G_T.dot(y)
        y_new = self.G.dot(x)
        return x_new/la.norm(x_new), y_new/la.norm(y_new)
    
    def get_scores(self):
        """Get the similarity scores to given tolerance or maxiters."""
        
        self._initialize()
        self.x, self.y = self._update(self.x0, self.y0)
        
        # Eventually fix this to be tolerance based
        for i in xrange(self.maxiters):
            self.x, self.y = self._update(self.x, self.y)
            
    
    def display_scores(self, show_graphs=False):
        """
        Display similarity scores and contextual information from graphs
        in an intuitive way.
        """
        if show_graphs:
            nx.draw_shell(self.G_A, with_labels=True, node_color='c')
            plt.show()

            nx.draw_shell(self.G_B, with_labels=True, node_color='c')
            plt.show()
        
        #print "\nNode Similarity Scores:"
        print np.around(self.x.reshape((self.n_B,self.n_A)),2)
        
        #print "\nEdge Similarity Scores:\n"
        #print np.around(self.y.reshape((self.m_B,self.m_A)),2)

In [143]:
G1 = nx.DiGraph()
G1.add_edges_from([(1,3),(3,1),(2,1),(2,3),(4,2),(3,4),(4,1)])

G2 = nx.DiGraph()
G2.add_edges_from([(1,3),(3,1),(2,1),(4,2),(3,4),(4,1)])

G3 = nx.DiGraph()
G3.add_edges_from([(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)])

G4 = nx.DiGraph()
G4.add_edges_from([(1,2),(2,3)])

G5 = nx.DiGraph()
G5.add_edges_from([(1,2),(2,3),(3,4),(4,5),(5,6)])

G6 = nx.DiGraph()
G6.add_edges_from([(1,2),(1,3),(1,4),(1,5),(1,6)])

G7 = nx.DiGraph()
G7.add_edges_from(itertools.permutations(range(1,5),2))

G8 = nx.DiGraph()
G8.add_edges_from(itertools.permutations(range(1,9),2))

graphs = [G1, G2, G3, G4, G5, G6, G7, G8]
graph_names = [str("G")+str(i) for i in xrange(1,9)]


for i in xrange(8):
    for j in xrange(i,8):
        print "\nCompare ", graph_names[i], " and ", graph_names[j], ":"
        S = simscore(graphs[i], graphs[j], maxiters=50)
        S.get_scores()
        S.display_scores(show_graphs=False)


Compare  G1  and  G1 :
[[ 0.6   0.13  0.28  0.14]
 [ 0.13  0.19  0.2   0.16]
 [ 0.28  0.2   0.37  0.2 ]
 [ 0.14  0.16  0.2   0.16]]

Compare  G1  and  G2 :
[[ 0.73  0.1   0.13  0.14]
 [ 0.12  0.12  0.19  0.18]
 [ 0.25  0.14  0.31  0.21]
 [ 0.13  0.11  0.19  0.18]]

Compare  G1  and  G3 :
[[ 0.07  0.17  0.31  0.61]
 [ 0.28  0.22  0.15  0.06]
 [ 0.24  0.24  0.23  0.2 ]
 [ 0.25  0.19  0.14  0.08]]

Compare  G1  and  G4 :
[[ 0.1   0.52  0.36  0.27]
 [ 0.29  0.05  0.2   0.48]
 [ 0.22  0.22  0.22  0.1 ]]

Compare  G1  and  G5 :
[[ 0.04  0.21  0.33  0.36]
 [ 0.28  0.13  0.09  0.19]
 [ 0.22  0.19  0.1   0.02]
 [ 0.08  0.23  0.33  0.34]
 [ 0.25  0.08  0.08  0.16]
 [ 0.2   0.19  0.13  0.04]]

Compare  G1  and  G6 :
[[ 0.04  0.17  0.17  0.17]
 [ 0.17  0.17  0.54  0.05]
 [ 0.05  0.05  0.05  0.05]
 [ 0.51  0.06  0.06  0.06]
 [ 0.06  0.06  0.51  0.05]
 [ 0.05  0.05  0.05  0.05]]

Compare  G1  and  G7 :
[[ 0.3   0.3   0.3   0.3 ]
 [ 0.19  0.19  0.19  0.19]
 [ 0.3   0.3   0.3   0.3 ]
 [ 0.19  0.19  0

In [144]:
def create_large_random():
    G = nx.DiGraph()
    n = 25
    m = 500
    G.add_edges_from([(np.random.randint(n),np.random.randint(n)) for i in xrange(m)])

    G.remove_edges_from(G.selfloop_edges())

    A = nx.incidence_matrix(G, oriented=True)
    print A.shape
    print sparse.isspmatrix_csc(A)
    
def test_abs():
    print "Test the addition/abs method:\n"
    %timeit np.abs(A)
    B = np.abs(A)
    %timeit A + B
    C = A + B
    %timeit 0.5*C
    %timeit 0.5*(A + np.abs(A))
    
def test_maxmin():
    print "\n Test the max/min method:\n"
    %timeit sparse.csc_matrix(A.shape)
    B = sparse.csc_matrix(A.shape)
    %timeit A.maximum(B)
    %timeit A.maximum(sparse.csr_matrix(A.shape))

In [152]:
C = np.array([[3,-1,-2,0,0,0],[-1,4,-3,0,0,0],[-2,-3,10,-5,0,0],[0,0,-5,9,0,-3],[0,0,0,0,3,-2],[0,0,0,-3,-2,5]])
print C
print la.inv(C)
print la.inv(C)[0,:]
print np.dot(la.inv(C)[0,:],np.array([3,4,10.,9,3,5]))

[[ 3 -1 -2  0  0  0]
 [-1  4 -3  0  0  0]
 [-2 -3 10 -5  0  0]
 [ 0  0 -5  9  0 -3]
 [ 0  0  0  0  3 -2]
 [ 0  0  0 -3 -2  5]]
[[ 1.21069519  0.93796791  0.84705882  0.64705882  0.35294118  0.52941176]
 [ 0.93796791  1.1197861   0.84705882  0.64705882  0.35294118  0.52941176]
 [ 0.84705882  0.84705882  0.84705882  0.64705882  0.35294118  0.52941176]
 [ 0.64705882  0.64705882  0.64705882  0.64705882  0.35294118  0.52941176]
 [ 0.35294118  0.35294118  0.35294118  0.35294118  0.64705882  0.47058824]
 [ 0.52941176  0.52941176  0.52941176  0.52941176  0.47058824  0.70588235]]
[ 1.21069519  0.93796791  0.84705882  0.64705882  0.35294118  0.52941176]
25.3839572193
